In [1]:
from libsbml import * 
reader = SBMLReader()
document = reader.readSBMLFromFile('created_PA14_after_model_polisher.xml')
model = document.getModel()

In [2]:
from Bio.KEGG import REST

In [3]:
reaction_list = model.getListOfReactions()

### Testing

In [4]:
reaction_1 =reaction_list[1]
res = reaction_1.getCVTerms().get(0).getResources()
res_length = res.getLength()
for i in range(res_length):
    value = res.getValue(i)
    #print(value)
    if 'kegg' in value:
        print(res.getValue(i))
        split = value.split('/')
        kegg_id = split[-1]
        print(kegg_id)

http://identifiers.org/kegg.reaction/R03857
R03857


In [5]:
result = REST.kegg_get('R03857').read()
#print(result)

In [6]:
splitted = result.split('\n')
for element in splitted:
    if 'rn' in element:
        sec_split = element.split()
        if 'PATHWAY' in element:
            pathway_map = 'pau'+ sec_split[1][2:]
        else:
            pathway_map = 'pau'+ sec_split[0][2:]
        print(pathway_map)

pau00071
pau01100
pau01212


# Add all pathways to a reaction

In [8]:
def get_KEGG_ID(reaction):
    kegg_id = ''
    term = reaction.getCVTerms()
    if term != None:
        res = term.get(0).getResources()
        res_length = res.getLength()
        for i in range(res_length):
            value = res.getValue(i)
            #print(value)
            if 'kegg' in value:
                #print(res.getValue(i))
                split = value.split('/')
                kegg_id = split[-1]
                return(kegg_id)
                #print(kegg_id)

In [9]:
def write_cv_term(react, pathway_id):
    cv = CVTerm()
    cv.setQualifierType(BIOLOGICAL_QUALIFIER)
    cv.setBiologicalQualifierType(BQB_OCCURS_IN)
    url = 'http://identifiers.org/kegg.pathway/'+pathway_id
    cv.addResource(url)#'http://identifiers.org/kegg.pathway/'+pathway_id)
    #print(cv)        
    
    status = react.addCVTerm(cv)
    
    
    if status == LIBSBML_OPERATION_SUCCESS:
        success += 1
    #    print('success')
    #elif status == LIBSBML_INVALID_OBJECT:
    #    print('invalid object')
    #elif status == LIBSBML_OPERATION_FAILED:
    #    print('failed')
    else:
        print(react, val)

In [10]:
success = 0
for react in reaction_list:
    try:
        kegg_id = get_KEGG_ID(react)
        if kegg_id:
            result = REST.kegg_get(kegg_id).read()
            splitted = result.split('\n')
            for element in splitted:
                if 'rn' in element:
                    sec_split = element.split()
                    if 'PATHWAY' in element:
                        pathway_map = 'pau'+ sec_split[1][2:]
                    else:
                        pathway_map = 'pau'+ sec_split[0][2:]
                        
                    try:
                        #print(pathway_map)
                        pathway_result = REST.kegg_get(pathway_map)
                        #write_cv_term(react, pathway_map)
                        
                        cv = CVTerm()
                        cv.setQualifierType(BIOLOGICAL_QUALIFIER)
                        cv.setBiologicalQualifierType(BQB_OCCURS_IN)
                        url = 'http://identifiers.org/kegg.pathway/'+pathway_map
                        cv.addResource(url)#'http://identifiers.org/kegg.pathway/'+pathway_id)
                        #print(cv)   
                        
                        status = react.addCVTerm(cv)
                        #print(status)
                        if status == LIBSBML_OPERATION_SUCCESS:
                            success += 1
                        else:
                            print(react, val)
                            
                    except:
                        #print(pathway_map)
                        continue
        else:
            continue
    except:
        continue
                
print(success)

9


In [7]:
newdocument = model.getSBMLDocument()
writeSBMLToFile(newdocument,'PA14_reactions_pathways.xml') # 1 means success, 0 means failure

1